In [34]:
# %pip install pyinform -Uq

%reload_ext watermark
%watermark

import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing, model_selection, metrics

from tqdm import tqdm

import pyinform

from python.acnets.datasets import load_connectivity

Last updated: 2022-02-21T11:37:01.479387+01:00

Python implementation: CPython
Python version       : 3.9.10
IPython version      : 8.0.1

Compiler    : Clang 11.1.0 
OS          : Darwin
Release     : 21.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit



In [35]:
ds = xr.load_dataset('data/julia2018_resting/timeseries_dosenbach2010.nc')
X = ds['timeseries']

In [36]:
# @np.vectorize
# def vec_te(i, j):
#   return X_subj[i] + X_subj[j]

# def calculate_transfer_entropy(X):
#   results = []
#   for X_subj in tqdm(X.values):
#     for i in X_subj.shape[0]:
#       for j in X_subj.shape[0]:
#         pass

#     subj_TE = vec_te(i,j)
#     results.append(subj_TE)
#     return results
#   return np.array(results)

# calculate_transfer_entropy(X)[0]



In [37]:
# te.te_compute(X[0].values, X[0].values)
# np.meshgrid(np.arange(X.c))[0].shape, X[0].shape
# X.coords['subject']
# np.meshgrid(range(1,10), range(1,10))


In [39]:
xx = np.array([
  [0,1,2,1,1,0,0,0,0],
  [150,300,150,150,0,0,0,0,0],
  [-10,5,-10,-10,0,0,0,0,0],
])

xx = (xx - xx.mean(axis=1, keepdims=True)) / (xx.std(axis=1, keepdims=True) + sys.float_info.epsilon)

# xx = (xx - xx.mean(axis=1)) / xx.std(axis=1)

xx, *_ = pyinform.utils.bin_series(xx, b=101)


entropies = np.zeros((xx.shape[0], xx.shape[0]))

for i, source in enumerate(xx):
  for j, target in enumerate(xx):
    te = pyinform.transfer_entropy(source, target, k=1)
    entropies[i,j] = te
    print(source, target, te)

entropies

[ 15  58 100  58  58  15  15  15  15] [ 15  58 100  58  58  15  15  15  15] 0.0
[ 15  58 100  58  58  15  15  15  15] [ 58 100  58  58  15  15  15  15  15] 0.5943609377704335
[ 15  58 100  58  58  15  15  15  15] [ 0 82  0  0 54 54 54 54 54] 0.5943609377704335
[ 58 100  58  58  15  15  15  15  15] [ 15  58 100  58  58  15  15  15  15] 1.0
[ 58 100  58  58  15  15  15  15  15] [ 58 100  58  58  15  15  15  15  15] 0.0
[ 58 100  58  58  15  15  15  15  15] [ 0 82  0  0 54 54 54 54 54] 0.0
[ 0 82  0  0 54 54 54 54 54] [ 15  58 100  58  58  15  15  15  15] 1.0
[ 0 82  0  0 54 54 54 54 54] [ 58 100  58  58  15  15  15  15  15] 0.0
[ 0 82  0  0 54 54 54 54 54] [ 0 82  0  0 54 54 54 54 54] 0.0


array([[0.        , 0.59436094, 0.59436094],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ]])

In [41]:
X_te = []

for X_sub_xr in tqdm(X):
  X_sub = X_sub_xr.values
  X_sub -= X_sub.mean(axis=1, keepdims=True)
  X_sub /= X_sub.std(axis=1, keepdims=True) + sys.float_info.epsilon
  X_sub, *_ = pyinform.utils.bin_series(X_sub, b=101)
  
  subj_te = np.zeros((X_sub.shape[0], X_sub.shape[0]))

  for i, source in enumerate(X_sub):
    for j, target in enumerate(X_sub):
      subj_te[i,j] = pyinform.transfer_entropy(source, target, k=1)
      subj_te[i,j]

  X_te.append(subj_te)

X_te = np.stack(X_te)

100%|██████████| 32/32 [01:56<00:00,  3.63s/it]


[array([[0.        , 1.63962495, 1.62430128, ..., 1.72570381, 1.63850426,
         1.57016383],
        [1.61667615, 0.        , 1.65655935, ..., 1.74183284, 1.67076232,
         1.62463873],
        [1.64893422, 1.70414108, 0.        , ..., 1.74183284, 1.67076232,
         1.64076777],
        ...,
        [1.66506325, 1.70414108, 1.65655935, ..., 0.        , 1.65463329,
         1.6085097 ],
        [1.64893422, 1.70414108, 1.64043031, ..., 1.72570381, 0.        ,
         1.6024219 ],
        [1.61058835, 1.68801204, 1.65655935, ..., 1.69344574, 1.63241646,
         0.        ]]),
 array([[0.        , 1.44937721, 1.40111925, ..., 1.28132285, 1.5212531 ,
         1.50203306],
        [1.65824927, 0.        , 1.44950635, ..., 1.36805582, 1.5212531 ,
         1.50203306],
        [1.65824927, 1.49776431, 0.        , ..., 1.27128162, 1.5212531 ,
         1.53429112],
        ...,
        [1.60377437, 1.48163528, 1.33660312, ..., 0.        , 1.55351116,
         1.469775  ],
        [1.6